In [5]:
import json
import pickle

In [6]:
# Original
with open("original_results.json", "r") as file:
    data = json.load(file)

parsed_data = {}

for key, value in data.items():
    idx = int(key.split("/")[-1].split(".")[0])
    parsed_data[idx] = [[value['sigma_est_0']], [value['sigma_err_0']]]

for idx, values in sorted(parsed_data.items()):
    print(f"{idx} : {values}")

0 : [[0.208595], [0.0242589]]
1 : [[0.248679], [0.0222827]]
2 : [[0.247789], [0.0214003]]
3 : [[0.243477], [0.020656]]
4 : [[0.264021], [0.0221762]]
5 : [[0.232209], [0.0211401]]
6 : [[0.266406], [0.0230365]]
7 : [[0.268994], [0.0288309]]
8 : [[0.274601], [0.0242799]]
9 : [[0.25589], [0.0214221]]
10 : [[0.239299], [0.0251464]]
11 : [[0.204565], [0.0240014]]
12 : [[0.254956], [0.0211375]]
13 : [[0.268656], [0.0219735]]
14 : [[0.237235], [0.0244934]]
15 : [[0.205192], [0.0215834]]
16 : [[0.26636], [0.0225192]]
17 : [[0.259538], [0.0224781]]
18 : [[0.220604], [0.0201227]]
19 : [[0.23466], [0.0203024]]
20 : [[0.257095], [0.0242663]]
21 : [[0.257855], [0.024132]]
22 : [[0.262901], [0.0208686]]
23 : [[0.265225], [0.0202832]]
24 : [[0.270648], [0.0225688]]


In [7]:
file_path = "/u/home/j/jiayini/project-sriram/RHE_project/results/python_rhe_1.pkl"
with open(file_path, "rb") as file:
    content = pickle.load(file)

print(content)
# TODO: Parse
# TODO: multiple runs + plotting

{'/u/home/j/jiayini/project-sriram/RHE_project/data/pheno/bin_1/1.phen': {'sigma_ests_total': array([0.24853191, 0.74164602]), 'sig_errs': [0.021905915191001206, 0.02190591519100168], 'h2_total': array([0.25099722, 0.74900278, 0.25099722]), 'h2_errs': [0.022123211005491406, 0.02212321100549139, 0.022123211005491406], 'enrichment_total': array([1.10081967e-06]), 'enrichment_errs': [1.713688290370535e-11]}}
